In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Fraud Detection").getOrCreate()
df = spark.read.csv("transactions.csv", header=True, inferSchema=True)
df.show(5)

25/03/06 01:07:11 WARN Utils: Your hostname, bonaogeto-HP-EliteBook-x360-1040-G5 resolves to a loopback address: 127.0.1.1; using 192.168.0.103 instead (on interface wlp108s0)
25/03/06 01:07:11 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/06 01:07:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
                                                                                

+--------------+-------+-------+--------+--------------------+----------------+-------------+--------------+-----+
|transaction_id|user_id| amount|location|                time|transaction_type|       device|previous_fraud|fraud|
+--------------+-------+-------+--------+--------------------+----------------+-------------+--------------+-----+
|             1|  34340|4626.46|  Moscow|2024-08-26 13:27:...|      Withdrawal|       Mobile|             0|    1|
|             2|  31435|  71.81|   Tokyo|2020-12-14 10:14:...|        Transfer|Crypto Wallet|             0|    0|
|             3|  27691|9834.56|   Dubai|2023-01-12 07:08:...|    Bill Payment|       Mobile|             0|    0|
|             4|  15252|9932.46|   Tokyo|2024-08-05 09:53:...|      Withdrawal|       Mobile|             0|    1|
|             5|  24169| 517.29|   Dubai|2020-01-12 11:23:...|      Withdrawal|          ATM|             0|    1|
+--------------+-------+-------+--------+--------------------+----------------+-

In [2]:
df.filter(df["amount"] > 5000).show()

+--------------+-------+-------+-------------+--------------------+----------------+-------------+--------------+-----+
|transaction_id|user_id| amount|     location|                time|transaction_type|       device|previous_fraud|fraud|
+--------------+-------+-------+-------------+--------------------+----------------+-------------+--------------+-----+
|             3|  27691|9834.56|        Dubai|2023-01-12 07:08:...|    Bill Payment|       Mobile|             0|    0|
|             4|  15252|9932.46|        Tokyo|2024-08-05 09:53:...|      Withdrawal|       Mobile|             0|    1|
|             6|  26064|7641.76|    Las Vegas|2024-03-15 20:35:...|      Withdrawal|          POS|             0|    1|
|             8|  16757|7364.33|       Moscow|2021-10-25 06:28:...|    Bill Payment|       Mobile|             0|    1|
|             9|  36631|6881.56|       London|2021-12-25 16:55:...|        Purchase|Crypto Wallet|             0|    1|
|            10|  46028|7679.67|      Ch

In [3]:
df.groupBy("location").count().orderBy("count", ascending=False).show()

+-------------+-----+
|     location|count|
+-------------+-----+
|       London| 5126|
|     New York| 5082|
|        Dubai| 5035|
|        Miami| 5016|
|    Las Vegas| 5009|
|        Tokyo| 5003|
|  Los Angeles| 4966|
|       Moscow| 4945|
|      Chicago| 4921|
|San Francisco| 4897|
+-------------+-----+



In [4]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer, VectorAssembler

indexer = StringIndexer(inputCol="location", outputCol="location_indexed")
# df = indexer.fit(df).transform(df)
df.show()

+--------------+-------+-------+-----------+--------------------+----------------+-------------+--------------+-----+
|transaction_id|user_id| amount|   location|                time|transaction_type|       device|previous_fraud|fraud|
+--------------+-------+-------+-----------+--------------------+----------------+-------------+--------------+-----+
|             1|  34340|4626.46|     Moscow|2024-08-26 13:27:...|      Withdrawal|       Mobile|             0|    1|
|             2|  31435|  71.81|      Tokyo|2020-12-14 10:14:...|        Transfer|Crypto Wallet|             0|    0|
|             3|  27691|9834.56|      Dubai|2023-01-12 07:08:...|    Bill Payment|       Mobile|             0|    0|
|             4|  15252|9932.46|      Tokyo|2024-08-05 09:53:...|      Withdrawal|       Mobile|             0|    1|
|             5|  24169| 517.29|      Dubai|2020-01-12 11:23:...|      Withdrawal|          ATM|             0|    1|
|             6|  26064|7641.76|  Las Vegas|2024-03-15 2

In [5]:
assembler = VectorAssembler(inputCols=["amount", "location_indexed"], outputCol="features")
# df = assembler.transform(df)
df.show()

+--------------+-------+-------+-----------+--------------------+----------------+-------------+--------------+-----+
|transaction_id|user_id| amount|   location|                time|transaction_type|       device|previous_fraud|fraud|
+--------------+-------+-------+-----------+--------------------+----------------+-------------+--------------+-----+
|             1|  34340|4626.46|     Moscow|2024-08-26 13:27:...|      Withdrawal|       Mobile|             0|    1|
|             2|  31435|  71.81|      Tokyo|2020-12-14 10:14:...|        Transfer|Crypto Wallet|             0|    0|
|             3|  27691|9834.56|      Dubai|2023-01-12 07:08:...|    Bill Payment|       Mobile|             0|    0|
|             4|  15252|9932.46|      Tokyo|2024-08-05 09:53:...|      Withdrawal|       Mobile|             0|    1|
|             5|  24169| 517.29|      Dubai|2020-01-12 11:23:...|      Withdrawal|          ATM|             0|    1|
|             6|  26064|7641.76|  Las Vegas|2024-03-15 2

In [6]:
lr = LogisticRegression(featuresCol="features", labelCol="fraud")
model = lr.fit(df)

# predict fraud cases
predictions = model.transform(df)
predictions.select("fraud", "prediction").show()

IllegalArgumentException: features does not exist. Available: transaction_id, user_id, amount, location, time, transaction_type, device, previous_fraud, fraud

In [ ]:
df.show()

In [ ]:
from pyspark.sql.types import StructType, StructField, FloatType, IntegerType, StringType
from pyspark.sql import functions as F

schema = StructType([
    StructField("transaction_id", IntegerType()),
    StructField("user_id", IntegerType()),
    StructField("amount", FloatType()),
    StructField("location", StringType()),
    StructField("time", StringType())
])

streaming_df = spark.readStream.schema(schema).csv("streaming_data/")
predictions = model.transform(streaming_df)

query = predictions.writeStream.outputMode("append").format("console").start()
query.awaitTermination()